<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Import pandas here

# --- 1. Environmental & Global Parameters ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=100):
        self.total_sim_steps = total_sim_steps
        self.days_per_cycle = days_per_cycle # Steps per "day" cycle
        self.time = 0 # Current time step (index)

        # UV Profile (simplified sine wave for circadian cycle)
        # np.linspace creates an array with 'total_sim_steps' points, so indices 0 to total_sim_steps-1
        self.uv_intensity = np.sin(np.linspace(0, 2 * np.pi * (total_sim_steps / days_per_cycle), total_sim_steps))
        self.uv_intensity[self.uv_intensity < 0] = 0 # No negative UV, night is 0

    def get_current_uv(self):
        # Ensure we don't go out of bounds if time somehow exceeds max index
        if self.time >= self.total_sim_steps:
            return 0.0 # Return 0 if simulation time somehow exceeds array bounds
        return self.uv_intensity[self.time]

    def step(self):
        self.time += 1
        self.current_day_step = self.time % self.days_per_cycle # Corrected typo here

# --- 2. Proto-Cell/System Class ---
class ProtoCell:
    def __init__(self, env):
        self.env = env

        # Core Molecular Pools (as concentrations/amounts)
        self.rna_pool = 100.0 # General RNA (templates, proto-rRNA)
        self.dna_pool = 10.0  # Basic DNA (more stable template)
        self.atp_pool = 50.0  # Energy currency
        self.mcyw_peptide_pool = 5.0 # Foraging/Antenna peptides
        self.other_peptide_pool = 2.0 # Non-mcyw peptides

        # Resource Stocks
        self.internal_ntp_stock = 100.0
        self.internal_aa_stock = 100.0
        self.internal_trp_tyr_cys_stock = 10.0 # Foraged rare AAs

        # System States & Damage
        self.rna_damage_level = 0.0 # Accumulates from UV
        self.protein_synthesis_active = True # Whether ribosomes are active
        self.rna_dna_copying_active = True # Whether replication is active
        self.uv_safety_status = "SAFE" # Based on internal signals
        self.premature_stop_signal = 0.0 # Initialize here, updated in apply_uv_damage

        # --- Parameters (tune these to observe different behaviors) ---
        # UV Damage parameters
        self.uv_damage_rate_per_uv = 0.05 # How much UV damages RNA
        self.damage_tolerance_threshold = 5.0 # Max damage before arrest

        # Foraging parameters (mcyw efficiency, ATP cost, preference for rare AAs)
        self.foraging_efficiency_mcyw = 0.1 # Resource units per mcyw per step
        self.foraging_atp_cost = 0.01 # ATP consumed per foraging action
        self.trp_tyr_cys_foraging_bias = 5.0 # How much more mcyw prioritizes rare AAs

        # Protein Synthesis parameters
        self.protein_synthesis_rate = 0.1 # Peptide units per step (when active)
        self.protein_synthesis_ntp_cost = 0.1
        self.protein_synthesis_aa_cost = 0.1
        self.protein_synthesis_atp_cost = 0.02
        self.mcyw_synthesis_bias = 0.3 # Proportion of protein synthesis making mcyw

        # Stop Codon Precariousness/Signaling
        self.precarious_codon_rate_per_damage = 0.1 # How often damage leads to a premature stop
        self.premature_stop_signal_threshold = 0.5 # Level of signal to trigger arrest

        # ATP Synthesis (from mcyw antenna chains and UV)
        self.atp_generation_rate_per_mcyw_uv = 0.01 # ATP from mcyw in light

        # Repair parameters
        self.repair_rate = 0.05 # How much damage repaired per step
        self.repair_atp_cost = 0.01
        self.trp_tyr_cys_for_repair_needed = 0.1 # Trp/Tyr/Cys consumed for repair

        # RNA/DNA Copying parameters
        self.rna_dna_copy_rate = 0.01 # Rate of template copying
        self.rna_dna_copy_ntp_cost = 0.05
        self.rna_dna_copy_atp_cost = 0.01

        # Degradation rates
        self.rna_degradation_rate = 0.001
        self.peptide_degradation_rate = 0.002
        self.atp_degradation_rate = 0.005

    # --- Core Processes ---
    def _apply_uv_damage(self):
        current_uv = self.env.get_current_uv()
        if current_uv > 0:
            damage_increase = current_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            # Simulate the "safety reportage" signal from damaged precarious codons
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else:
            self.premature_stop_signal = 0 # No new stops at night

    def _forage(self):
        if self.mcyw_peptide_pool <= 0 or self.atp_pool <= self.foraging_atp_cost:
            return # Cannot forage without mcyw or ATP

        # Consume ATP for foraging movement
        atp_consumed = self.foraging_atp_cost * self.mcyw_peptide_pool
        self.atp_pool -= atp_consumed
        self.atp_pool = max(0, self.atp_pool) # Ensure not negative

        # Acquire general AAs and NTPs

In [ ]:
display(cell.__dict__)

NameError: name 'cell' is not defined

In [ ]:
# Run the simulation for a few steps and store the rna_pool values
total_sim_steps = 200 # Let's run for 200 steps
env = Environment(total_sim_steps=total_sim_steps)
proto_cell = ProtoCell(env)

rna_pool_history = []
time_steps = []

for step in range(total_sim_steps):
    # Assuming you have a step method in ProtoCell to advance the simulation
    # proto_cell.step() # Need to implement this in the ProtoCell class
    # For now, let's just record the current rna_pool and simulate a simple change
    rna_pool_history.append(proto_cell.rna_pool)
    time_steps.append(env.time)

    # Simulate some changes for demonstration (replace with actual step logic)
    current_uv = env.get_current_uv()
    proto_cell._apply_uv_damage() # Apply UV damage
    # Add other simulation steps here when implemented in ProtoCell

    # Increment environment time
    env.step()


# Plot the rna_pool over time
plt.figure(figsize=(10, 6))
plt.plot(time_steps, rna_pool_history)
plt.xlabel('Simulation Step')
plt.ylabel('RNA Pool')
plt.title('RNA Pool over Time')
plt.grid(True)
plt.show()

AttributeError: 'Environment' object has no attribute 'days_day_cycle'